# Column size check
Assigns maximum reinforcement to current columns

In [4]:
import pandas as pd
import numpy as np
import math
import lib.csiapi as csiapi

ETABSObject, SapModel = csiapi.get_active_etabs_object()

Succesfully created helper object
Succesfully created SapModel


## Insert parameters here

In [6]:
max_spacing_column = 0.04
max_percentage = 2.5

max_percentage_flag = True # set True if using max percentage instead of max_spacing

rebar_options = [22, 25]

25

In [60]:
# Set to kN, m, C
SapModel.SetPresentUnits_2(4,6,2)
# Get all column sections
column_sections = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Rectangular')
# column_sections_circular = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Circle')
column_reinforcement = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Column Reinforcing')

column_table = column_sections.merge(column_reinforcement, left_on='Name', right_on='Name')
# column_table = column_sections.merge(column_sections_circular, left_on='Name', right_on='Name')
column_table_summary = column_table.loc[:,'Name']

# Assign max reinforcement for every column
SapModel.SetModelIsLocked(False)
for i in column_table.iterrows():
    _, i = i
    name = i['Name']
    var1, var2, var3, var4, var5, var6, var7, var8, var9, var10, var11, var12, var13, var14, _, _, _, ret = SapModel.PropFrame.GetRebarColumn_1(name)
    '''
    7 = R3 bars
    8 = R2 bars
    9 = rebar size 
    14 = to be designed
    '''
    var9 = str(max(rebar_options))
    var14 = False
    if max_percentage_flag:
        area = i['Width'] * i['Depth']
        max_as = area * max_percentage / 100
        as_bar = (int(var9)/1000) ** 2 * math.pi / 4
        max_bars = math.floor(max_as / as_bar / 2) * 2
        var7 = (2 + max_bars / 2) / (1 + i['Depth'] / i['Width'])
        var8 = (i['Depth'] / i['Width']) * var7
        var7 = round(var7)
        var8 = round(var8)
        # print(max_bars,var7,var8)
        
    else:
        var7 = math.floor((i['Width'] - 0.1 + max_spacing_column) / (max_spacing_column + int(var9)/1000))
        var8 = math.floor((i['Depth'] - 0.1 + max_spacing_column) / (max_spacing_column + int(var9)/1000))

    # Assign columns 
    ret = SapModel.PropFrame.SetRebarColumn(name,var1,var2,var3,var4,var5,var6,var7,var8,var9,var10,var11,var12,var13,var14)

In [62]:
column_sections = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Rectangular')
# column_sections_circular = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Circle')
column_reinforcement = csiapi.get_table_display(SapModel, 'Frame Section Property Definitions - Concrete Column Reinforcing')

column_table = column_sections.merge(column_reinforcement, left_on='Name', right_on='Name')

column_table

,Name,Material,From File?,Depth,Width,Rigid Zone?,Notional Size Type,Notional Auto Factor,Design Type,Area Modifier,...,Clear Cover to Ties,Number Bars 3-Dir,Number Bars 2-Dir,Number Longitudinal Bars,Longitudinal Bar Size,Corner Bar Size,Tie Bar Size,Tie Bar Spacing,Number Ties 3-Dir,Number Ties 2-Dir
0,C10060,30MPa,No,1.000,0.600,No,Auto,1,Column,1,...,0.04,6.0,11.0,NaN,25,25.0,10,0.15,3.0,3.0
1,C3030,30MPa,No,0.300,0.300,No,Auto,1,Column,1,...,0.04,2.0,2.0,NaN,25,25.0,10,0.15,3.0,3.0
2,C5030,30MPa,No,0.300,0.500,No,Auto,1,Column,1,...,0.04,3.0,2.0,NaN,25,25.0,10,0.15,3.0,3.0
3,C5050,30MPa,No,0.500,0.500,No,Auto,1,Column,1,...,0.04,4.0,4.0,NaN,25,25.0,10,0.15,3.0,3.0
4,C5095,30MPa,No,0.500,0.950,No,Auto,1,Column,1,...,0.04,9.0,5.0,NaN,25,25.0,10,0.15,3.0,3.0
5,C60100,30MPa,No,0.600,1.000,No,Auto,1,Column,1,...,0.04,11.0,6.0,NaN,25,25.0,10,0.15,3.0,3.0
6,C6030,30MPa,No,0.600,0.300,No,Auto,1,Column,1,...,0.04,2.0,4.0,NaN,25,25.0,10,0.15,3.0,3.0
7,C9550,30MPa,No,0.950,0.500,No,Auto,1,Column,1,...,0.04,5.0,9.0,NaN,25,25.0,10,0.15,3.0,3.0
8,CB3060,30MPa,No,0.300,0.600,No,Auto,1,Column,1,...,0.04,4.0,2.0,NaN,25,25.0,10,0.15,3.0,3.0
9,CB5050,30MPa,No,0.500,0.500,No,Auto,1,Column,1,...,0.04,4.0,4.0,NaN,25,25.0,10,0.15,3.0,3.0
